In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import datetime
import math
import random
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from statistics import mean

In [125]:
x = [
    [1,2],
    [1,3],
    [2,3],
    [2,4],
    [3,2],
    [3,5],
    [4,1],

]
y = [1.03,-1.44,4.53,2.24,13.27,5.62,21.53]

In [3]:
def signo(x):
  if x>=0:
    return 1
  else:
    return -1

In [108]:
def predict(modelo,x):
  if type(x[0]) == list:
    pred_v = []
    for x_it in x:
      pred = modelo[len(modelo)-1]
      for i in range(len(modelo)-1):
        pred+= modelo[i]*x_it[i]
      pred_v.append(pred)
    return pred_v
  else:
    pred = modelo[len(modelo)-1]
    for i in range(len(modelo)-1):
      pred+= modelo[i]*x[i]

    return pred

def MAE(modelo,x,y):
  pred = predict(modelo,x)
  errs =[e1 - e2 for e1, e2 in zip(y,pred)]
  abs_v = [abs(ele) for ele in errs]

  return mean(abs_v)
def MSE(modelo,x,y):
  pred = predict(modelo,x)
  errs =[e1 - e2 for e1, e2 in zip(y,pred)]
  squ_v = [(ele)**2 for ele in errs]

  return mean(squ_v)

def MSE_GRAD(modelo,x,errs,w_it,l_rate):

  if w_it < len(modelo)-1:
        #len(modelo)-1 porque el último peso es el sesgo y se ajusta
        #de forma distinta
        #Aqui se ajustan todos los pesos menos el bias

        #Cálculo del Σ((Ŷ-Y)*Xji)
        errs_copy = errs.copy()
        for er in range(len(errs_copy)):
          errs_copy[er]= errs_copy[er]*x[er][w_it]
        modelo[w_it] = modelo[w_it]-1*(l_rate)*np.mean(errs_copy)

  else:
    #Aqui se ajusta el bias
    modelo[w_it] = modelo[w_it] -1*(2*l_rate*np.mean(errs) )

def MAE_GRAD(modelo,errs,w_it,l_rate):
  if w_it < len(modelo)-1:
        for er in range(len(errs)):
          errs[er]= signo(errs[er])*x[er][w_it]
        modelo[w_it] = modelo[w_it]-1*(l_rate)*mean(errs)
        #Aqui se ajusta el bias
  else:
    errs = [signo(x) for x in errs]
    modelo[w_it] = modelo[w_it] -1*(l_rate*mean(errs) )

In [5]:
predict([1,1,1],x)

[4, 5, 6, 7, 6, 9, 6, 4, 5, 6, 7, 6, 9, 6]

In [6]:
modelo = [0,0,0]
#[x1,x2,b]

In [136]:
def linear_regression(modelo,x,y,n_it,l_rate,func_ajuste):
  for i in range(n_it):
    errs = []
    pred = predict(modelo,x)
    errs = [e1 - e2 for e1, e2 in zip(pred,y)]

    for w_it in range(len(modelo)):
      errs_it = errs.copy()
      func_ajuste(modelo,x,errs_it,w_it,l_rate)
  return modelo

In [ ]:
modelo = [1,1,1]
print('Ajuste con MSE')
print('Error inicial -> ',MSE(modelo,x,y))
linear_regression(modelo,x,y,20000,0.001,MSE_GRAD)
print('Error final -> ',MSE(modelo,x,y))
print(modelo)

modelo = [1,1,1]
print('\nAjuste con MAE')
print('Error inicial -> ',MAE(modelo,x,y))
linear_regression(modelo,x,y,20000,0.001,MAE_GRAD)
print('Error final -> ',MAE(modelo,x,y))
print(modelo)

In [21]:
def generate_mini_batch(x, y, batch_percentage):
  tam_mini_batch = math.trunc(len(x)*batch_percentage)
  print(tam_mini_batch)
  mini_batch_x = []
  mini_batch_y = []
  x_copy = x.copy()
  y_copy = y.copy()

  while len(x_copy)>0:
    it_mini_batchx = []
    it_mini_batchy = []
    while len(it_mini_batchx)<tam_mini_batch:
      index = random.randint(0,len(x_copy)-1)
      it_mini_batchx.append(x_copy[index])
      it_mini_batchy.append(y_copy[index])
      x_copy.pop(index)
      y_copy.pop(index)
    mini_batch_x.append(it_mini_batchx)
    mini_batch_y.append(it_mini_batchy)

  return mini_batch_x,mini_batch_y


In [76]:
data = pd.read_csv('diabetes.tab.csv',delimiter='\t')

In [12]:
print(data)

     AGE  SEX   BMI      BP   S1     S2    S3    S4      S5   S6    Y
0     59    2  32.1  101.00  157   93.2  38.0  4.00  4.8598   87  151
1     48    1  21.6   87.00  183  103.2  70.0  3.00  3.8918   69   75
2     72    2  30.5   93.00  156   93.6  41.0  4.00  4.6728   85  141
3     24    1  25.3   84.00  198  131.4  40.0  5.00  4.8903   89  206
4     50    1  23.0  101.00  192  125.4  52.0  4.00  4.2905   80  135
..   ...  ...   ...     ...  ...    ...   ...   ...     ...  ...  ...
437   60    2  28.2  112.00  185  113.8  42.0  4.00  4.9836   93  178
438   47    2  24.9   75.00  225  166.0  42.0  5.00  4.4427  102  104
439   60    2  24.9   99.67  162  106.6  43.0  3.77  4.1271   95  132
440   36    1  30.0   95.00  201  125.2  42.0  4.79  5.1299   85  220
441   36    1  19.6   71.00  250  133.2  97.0  3.00  4.5951   92   57

[442 rows x 11 columns]


In [128]:
def from_dataset_to_arrays(data):
  Y = data['Y']
  y = list(Y.values)
  data = data.drop(columns='Y')
  x = list(data.values)
  for a in range(len(x)):
    b = list(x[a])
    x[a] = b

  return x,y
x,y = from_dataset_to_arrays(data)
print(x)
print(y)


[[59.0, 2.0, 32.1, 101.0, 157.0, 93.2, 38.0, 4.0, 4.8598, 87.0], [48.0, 1.0, 21.6, 87.0, 183.0, 103.2, 70.0, 3.0, 3.8918, 69.0], [72.0, 2.0, 30.5, 93.0, 156.0, 93.6, 41.0, 4.0, 4.6728, 85.0], [24.0, 1.0, 25.3, 84.0, 198.0, 131.4, 40.0, 5.0, 4.8903, 89.0], [50.0, 1.0, 23.0, 101.0, 192.0, 125.4, 52.0, 4.0, 4.2905, 80.0], [23.0, 1.0, 22.6, 89.0, 139.0, 64.8, 61.0, 2.0, 4.1897, 68.0], [36.0, 2.0, 22.0, 90.0, 160.0, 99.6, 50.0, 3.0, 3.9512, 82.0], [66.0, 2.0, 26.2, 114.0, 255.0, 185.0, 56.0, 4.55, 4.2485, 92.0], [60.0, 2.0, 32.1, 83.0, 179.0, 119.4, 42.0, 4.0, 4.4773, 94.0], [29.0, 1.0, 30.0, 85.0, 180.0, 93.4, 43.0, 4.0, 5.3845, 88.0], [22.0, 1.0, 18.6, 97.0, 114.0, 57.6, 46.0, 2.0, 3.9512, 83.0], [56.0, 2.0, 28.0, 85.0, 184.0, 144.8, 32.0, 6.0, 3.5835, 77.0], [53.0, 1.0, 23.7, 92.0, 186.0, 109.2, 62.0, 3.0, 4.3041, 81.0], [50.0, 2.0, 26.2, 97.0, 186.0, 105.4, 49.0, 4.0, 5.0626, 88.0], [61.0, 1.0, 24.0, 91.0, 202.0, 115.4, 72.0, 3.0, 4.2905, 73.0], [34.0, 2.0, 24.7, 118.0, 254.0, 184.2, 39

In [139]:
modelo = []
for i in range(len(x[0])):
  modelo.append(1)
print(len(y))
print(type(x[0]))
print('Ajuste con MSE')
print('Error inicial -> ',MAE(modelo,x,y))
modelo =linear_regression(modelo,x,y,10000,0.00001,MSE_GRAD)
print('Error final -> ',MAE(modelo,x,y))
print(modelo)


442
<class 'list'>
Ajuste con MSE
Error inicial ->  382.9552796380091
Error final ->  46.61239230730113
[-0.08925189967029973, 0.3876218006975746, 4.578874013689941, 0.8283779719596283, 1.3824956163251279, -1.5655391426204623, -2.683542168775894, 1.0187184588578593, 1.1599596164282788, 0.846327005385191]


In [142]:
predict(modelo,x[0])

205.88474637028582

In [144]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error



# Suponiendo que 'x' es tu conjunto de características y 'y' es tu variable objetivo

# Crear un modelo de regresión lineal
modelo_regresion = LinearRegression()

# Entrenar el modelo con los datos
modelo_regresion.fit(x, y)

# Una vez entrenado, puedes usar el modelo para hacer predicciones
predicciones = modelo_regresion.predict(x)
# Calcular el MSE del modelo
mae = mean_absolute_error(y, predicciones)

print("Error Cuadrático Medio (MSE):", mae)

Error Cuadrático Medio (MSE): 43.27745202531506


In [146]:
modelo_regresion.predict([x[0]])

array([206.11667725])